In [1]:
!pip install /kaggle/input/libraries/joblib-1.4.2-py3-none-any.whl
!pip install /kaggle/input/libraries/lightgbm-4.3.0-py3-none-manylinux_2_28_x86_64.whl
!pip install /kaggle/input/libraries/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/libraries/optuna-3.5.0-py3-none-any.whl
!pip install /kaggle/input/libraries/pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/libraries/scikit_learn-1.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/libraries/torch-2.2.2-cp310-cp310-manylinux1_x86_64.whl
!pip install /kaggle/input/libraries/scikit_learn-1.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/libraries/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/libraries/joblib-1.4.2-py3-none-any.whl
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully uninstalled joblib-1.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
Processing /kaggle/input/libraries/lightgbm-4.3.0-py3-none-manylinux_2_28_x86_64.whl
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, Descriptors, DataStructs
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import optuna
import torch
import torch.nn as nn
import torch.utils.data
import joblib


train_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

def featurize_combo(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2048 + 167)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    morgan_arr = np.zeros((2048,), dtype=int)
    DataStructs.ConvertToNumpyArray(morgan_fp, morgan_arr)
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    maccs_arr = np.zeros((167,), dtype=int)
    DataStructs.ConvertToNumpyArray(maccs_fp, maccs_arr)
    return np.concatenate([morgan_arr, maccs_arr])

def calc_extended_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(12)
    desc = [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.RingCount(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.FpDensityMorgan1(mol),
        Descriptors.FpDensityMorgan2(mol),
        Descriptors.FpDensityMorgan3(mol),
        Descriptors.HeavyAtomCount(mol),
        Descriptors.NumValenceElectrons(mol),
    ]
    return np.array(desc)

print("Featurizing train data...")
fps_train = [featurize_combo(smi) for smi in train_df['SMILES']]
desc_train = [calc_extended_descriptors(smi) for smi in train_df['SMILES']]
X_train_full = np.hstack([fps_train, desc_train])

print("Featurizing test data...")
fps_test = [featurize_combo(smi) for smi in test_df['SMILES']]
desc_test = [calc_extended_descriptors(smi) for smi in test_df['SMILES']]
X_test_full = np.hstack([fps_test, desc_test])



feature_scaler = StandardScaler()
feature_scaler.fit(X_train_full)
joblib.dump(feature_scaler, 'feature_scaler.pkl')
print("Saved global feature scaler as feature_scaler.pkl")

X_train_scaled = feature_scaler.transform(X_train_full)
X_test_scaled = feature_scaler.transform(X_test_full)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim=1, hidden_dims=[512, 128], dropout_rates=[0.3, 0.2]):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hdim, drop in zip(hidden_dims, dropout_rates):
            layers.append(nn.Linear(prev_dim, hdim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(drop))
            prev_dim = hdim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def train_nn(X_train, y_train, X_val, y_val, params, epochs=30, batch_size=64):
    model = SimpleNN(X_train.shape[1], output_dim=1, hidden_dims=params['hidden_dims'], dropout_rates=params['dropout_rates']).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
    loss_fn = nn.MSELoss()

    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                   torch.tensor(y_train, dtype=torch.float32).unsqueeze(1))
    val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                                 torch.tensor(y_val, dtype=torch.float32).unsqueeze(1))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = float('inf')
    best_model_state = None
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                loss = loss_fn(preds, yb)
                val_losses.append(loss.item())
        avg_val_loss = np.mean(val_losses)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
    model.load_state_dict(best_model_state)
    return model


def tune_lgbm(X, y):
    def objective(trial):
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 31, 256),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
            'seed': 42,
            'verbose': -1,
        }
        cv = KFold(n_splits=3, shuffle=True, random_state=42)
        rmses = []
        for train_idx, val_idx in cv.split(X):
            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y[train_idx], y[val_idx]
            train_data = lgb.Dataset(X_tr, label=y_tr)
            val_data = lgb.Dataset(X_val, label=y_val)
            model = lgb.train(
                params,
                train_data,
                valid_sets=[val_data],
                num_boost_round=1000,
                callbacks=[
                    lgb.early_stopping(50),
                    lgb.log_evaluation(0)
                ],
            )
            preds = model.predict(X_val, num_iteration=model.best_iteration)
            rmse = np.sqrt(mean_squared_error(y_val, preds))
            rmses.append(rmse)
        return np.mean(rmses)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=20, show_progress_bar=True)
    return study.best_params

def tune_nn_params(X_train, y_train, X_val, y_val):
    def objective(trial):
        params = {
            'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
            'hidden_dims': [trial.suggest_int('hd1', 256, 512), trial.suggest_int('hd2', 64, 256)],
            'dropout_rates': [trial.suggest_float('drop1', 0.1, 0.5), trial.suggest_float('drop2', 0.1, 0.5)],
        }
        model = train_nn(X_train, y_train, X_val, y_val, params, epochs=10)
        model.eval()
        with torch.no_grad():
            preds = model(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy().reshape(-1)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=15, show_progress_bar=True)
    return study.best_params


models = {}
nn_models = {}
scalers = {}

print("\n--- Training models per target ---")
for target in targets:
    print(f"\nProcessing target: {target}")
    mask = train_df[target].notnull()
    X_target = X_train_scaled[mask.values]
    y_target = train_df.loc[mask, target].values


    print("Tuning LightGBM...")
    best_params = tune_lgbm(X_target, y_target)
    best_params.update({'objective': 'regression', 'metric': 'rmse', 'verbose': -1, 'seed': 42})
    train_data = lgb.Dataset(X_target, label=y_target)
    model_lgb = lgb.train(best_params, train_data, num_boost_round=1000)
    models[target] = model_lgb

    print("Tuning Neural Network...")
    X_tr_nn, X_val_nn, y_tr_nn, y_val_nn = train_test_split(X_target, y_target, test_size=0.2, random_state=42)
    best_nn_params = tune_nn_params(X_tr_nn, y_tr_nn, X_val_nn, y_val_nn)
    print(f"Training NN with best params: {best_nn_params}")
    model_nn = train_nn(
        X_tr_nn, y_tr_nn, X_val_nn, y_val_nn,
        params={
            'lr': best_nn_params['lr'],
            'hidden_dims': [best_nn_params['hd1'], best_nn_params['hd2']],
            'dropout_rates': [best_nn_params['drop1'], best_nn_params['drop2']]
        },
        epochs=20
    )
    nn_models[target] = model_nn


print("\n--- Preparing stacking training data ---")
train_preds = pd.DataFrame(index=train_df.index, columns=targets)

for target in targets:
    mask = train_df[target].notnull()
    X_target = X_train_scaled[mask.values]
    model_lgb = models[target]
    model_nn = nn_models[target]

    preds_lgb = np.full(train_df.shape[0], np.nan)
    preds_lgb[mask.values] = model_lgb.predict(X_target)

    model_nn.eval()
    with torch.no_grad():
        preds_nn = np.full(train_df.shape[0], np.nan)
        preds_nn[mask.values] = model_nn(torch.tensor(X_target, dtype=torch.float32).to(device)).cpu().numpy().reshape(-1)

    avg_preds = np.nanmean(np.vstack([preds_lgb, preds_nn]), axis=0)
    train_preds[target] = avg_preds


train_preds_filled = train_preds.fillna(train_df[targets].median())


stack_model = LinearRegression()
stack_model.fit(train_preds_filled.values, train_df[targets].fillna(train_df[targets].median()).values)


final_train_preds = stack_model.predict(train_preds_filled.values)
true_train_targets = train_df[targets].fillna(train_df[targets].median()).values

rmses = np.sqrt(np.mean((true_train_targets - final_train_preds) ** 2, axis=0))
r2s = [r2_score(true_train_targets[:, i], final_train_preds[:, i]) for i in range(len(targets))]

print("\n=== Final Model Performance on Training Data ===")
for i, target in enumerate(targets):
    print(f"{target}: RMSE = {rmses[i]:.4f}, R2 = {r2s[i]:.4f}")
print(f"Overall RMSE (mean across targets): {rmses.mean():.4f}")
print(f"Overall R2 (mean across targets): {np.mean(r2s):.4f}")


class FullStackedModel:
    def __init__(self, models, nn_models, stack_model, feature_scaler, targets):
        self.models = models
        self.nn_models = nn_models
        self.stack_model = stack_model
        self.feature_scaler = feature_scaler
        self.targets = targets
        self.device = device

    def predict(self, X_raw):
        X_scaled = self.feature_scaler.transform(X_raw)
        preds_list = []
        for target in self.targets:
            model = self.models[target]
            model_nn = self.nn_models[target]

            lgb_pred = model.predict(X_scaled)

            model_nn.eval()
            with torch.no_grad():
                nn_pred = model_nn(torch.tensor(X_scaled, dtype=torch.float32).to(self.device)).cpu().numpy().reshape(-1)

            preds_avg = (lgb_pred + nn_pred) / 2
            preds_list.append(preds_avg)

        preds_stack_input = np.vstack(preds_list).T  
        final_preds = self.stack_model.predict(preds_stack_input)
        return final_preds


full_model = FullStackedModel(models, nn_models, stack_model, feature_scaler, targets)


joblib.dump(full_model, 'full_stacked_model.pkl')
print("Saved full combined model as full_stacked_model.pkl")

def create_submission(test_df, model, scaler, submission_path='submission.csv'):
    print("Featurizing test data for submission...")
    fps_test = [featurize_combo(smi) for smi in test_df['SMILES']]
    desc_test = [calc_extended_descriptors(smi) for smi in test_df['SMILES']]
    X_test_full = np.hstack([fps_test, desc_test])

    print("Predicting test data targets...")
    preds = model.predict(X_test_full) 

    submission_df = pd.DataFrame({
        'id': test_df['id'],
        'Tg': preds[:, 0],
        'FFV': preds[:, 1],
        'Tc': preds[:, 2],
        'Density': preds[:, 3],
        'Rg': preds[:, 4]
    })

    submission_df.to_csv(submission_path, index=False)
    print(f"Submission file saved as {submission_path}")


create_submission(test_df, full_model, feature_scaler)

Featurizing train data...


[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerator
[19:01:56] DEPRECATION WARNING: please use MorganGenerat

Featurizing test data...


[19:02:23] DEPRECATION WARNING: please use MorganGenerator
[19:02:23] DEPRECATION WARNING: please use MorganGenerator
[19:02:23] DEPRECATION WARNING: please use MorganGenerator
[I 2025-07-22 19:02:23,377] A new study created in memory with name: no-name-de532dc3-8047-42c0-8c26-f5f15b64c1ce


Saved global feature scaler as feature_scaler.pkl

--- Training models per target ---

Processing target: Tg
Tuning LightGBM...


...

  0%|          | 0/15 [00:00<?, ?it/s]

[I 2025-07-22 19:16:41,516] Trial 0 finished with value: 5.954339872851981 and parameters: {'lr': 0.004535874242635379, 'hd1': 431, 'hd2': 118, 'drop1': 0.20215299916543522, 'drop2': 0.19671013304498974}. Best is trial 0 with value: 5.954339872851981.
[I 2025-07-22 19:16:42,080] Trial 1 finished with value: 5.734598499459601 and parameters: {'lr': 0.005973819670098687, 'hd1': 286, 'hd2': 219, 'drop1': 0.4422847553525948, 'drop2': 0.23347861549102264}. Best is trial 1 with value: 5.734598499459601.
[I 2025-07-22 19:16:42,640] Trial 2 finished with value: 5.799972708742293 and parameters: {'lr': 0.0005924538346884702, 'hd1': 278, 'hd2': 148, 'drop1': 0.4643820687187411, 'drop2': 0.4527816106467245}. Best is trial 1 with value: 5.734598499459601.
[I 2025-07-22 19:16:43,421] Trial 3 finished with value: 6.0582471875370985 and parameters: {'lr': 0.009724928426603434, 'hd1': 506, 'hd2': 146, 'drop1': 0.4213145441532521, 'drop2': 0.48247486488785685}. Best is trial 1 with value: 5.73459849945

/tmp/ipykernel_13/2477270265.py:246: RuntimeWarning: Mean of empty slice
  avg_preds = np.nanmean(np.vstack([preds_lgb, preds_nn]), axis=0)



=== Final Model Performance on Training Data ===
Tg: RMSE = 6.4099, R2 = 0.9500
FFV: RMSE = 0.0074, R2 = 0.9298
Tc: RMSE = 0.0062, R2 = 0.9497
Density: RMSE = 0.0121, R2 = 0.9156
Rg: RMSE = 0.4542, R2 = 0.8832
Overall RMSE (mean across targets): 1.3780
Overall R2 (mean across targets): 0.9257
Saved full combined model as full_stacked_model.pkl
Featurizing test data for submission...
Predicting test data targets...
Submission file saved as submission.csv


[19:16:54] DEPRECATION WARNING: please use MorganGenerator
[19:16:54] DEPRECATION WARNING: please use MorganGenerator
[19:16:54] DEPRECATION WARNING: please use MorganGenerator
